In [ ]:
"""
Consolidates everything from all prior tutorials, except "euclidian stuff" (for that see 240410_kedar_euclidia...)
"""

In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt

# Load DFallPa dataset

In [ ]:

# this is the path to the dataset
path = f"/home/lucas/Dropbox/SCIENCE/FREIWALD_LAB/DATA/Xuan/DFallpa-Diego-230615-trial-kilosort_if_exists-norm=None-combine=True.pkl"
DFallpa = pd.read_pickle(path)


In [ ]:
pa = DFallpa["pa"].values[0]
dflab = pa.Xlabels["trials"]
dflab["taskconfig_shploc_SHSEM"].value_counts()
#  'taskconfig_loc',
#  'taskconfig_shp',
#  'taskconfig_shp_SHSEM',
#  'taskconfig_shploc',
#  'taskconfig_shploc_SHSEM',


In [3]:
##### Method: loading functrion"

In [4]:
from neuralmonkey.classes.population_mult import load_handsaved_wrapper, dfpa_match_chans_across_pa_each_bregion
from neuralmonkey.classes.population_mult import extract_single_pa


In [ ]:
# Method 1 - load a single DFallPA
DFallpa = load_handsaved_wrapper()


In [ ]:
# Method 1 - load a single DFallPA

# animal = "Diego"
# date = 240625

# animal = "Pancho"
# date = 240509
# version = "trial"
# combine = True
# question = "SP_BASE_trial"
animal = "Diego"
date = 230630   
version = "saccade_fix_on"
combine = True
question = None
ignore_question = True
DFallpa = load_handsaved_wrapper(animal=animal, date=date, combine_areas=combine, version=version, question=question, 
                                 ignore_question=ignore_question)


In [ ]:
DFallpa

In [ ]:
pa = DFallpa["pa"].values[1]
dflab = pa.Xlabels["trials"]
dflab["event_idx_within_trial"].value_counts()

In [ ]:
dflab["task_kind"].value_counts(0)

In [ ]:
dfl

In [ ]:
dflab["trialcode"].value_counts()

In [ ]:
print("\nTask kind...")
print(dflab[dflab["trialcode"] == "230630-1-564"]["task_kind"])

print("\nData for each fixation on this trial (during samp):")
dflab[dflab["trialcode"] == "230630-1-564"].loc[:, ["event_idx_within_trial", "shape-fixation", "loc-fixation", "seqc_0_shape", "seqc_0_loc"]]

In [ ]:
dflab[dflab["trialcode"] == "230630-1-564"]["shape-fixation"]

# Quick look at DFallpa

In [ ]:
# This holds data across multiple brain regions (the "bregion" column)
# -- "event", the event during trial which this data is aligned to
# -- "twind", time extracted aligned to that event, in seconds
# -- pa, PopAnal class holding neurla data and behavioral features.
# -- pa_x_shape, shape of neural data, in (chans, trials, times)
# Ingore the "which_level"

display(DFallpa)


In [ ]:
# Pull out a single pa
pa = DFallpa["pa"].values[17]


In [ ]:
# This holds data for a single brain area
# The main data is here, in pa.X, an array with dimensions (channels, trials, timepoints)
print(pa.X.shape)

# The labels for channels, trials, and times:
print("channels: ", pa.Chans)
print("trials: ", pa.Trials)
print("timepoints: ", pa.Times)




In [ ]:
# Each trials has associated "features", saved here, where each row of this dataframe matches a trial
pa.Xlabels["trials"]


In [ ]:
# e.g., to get the features for trial i, do this
trial = 10
features = pa.Xlabels["trials"].iloc[trial]
display(features)

# Preprocess methods

### [OPTIONAWrapper to do all steps

In [ ]:
from neuralmonkey.classes.population_mult import dfpa_concatbregion_preprocess_wrapper
fr_mean_subtract_method = "across_time_bins"
# fr_mean_subtract_method = "each_time_bin"
dfpa_concatbregion_preprocess_wrapper(DFallpa, fr_mean_subtract_method=fr_mean_subtract_method)

### (1a) FR normalization

##### Method 1 - each PA normalize independently

In [ ]:
from neuralmonkey.classes.population_mult import dfallpa_preprocess_fr_normalization
# fr_normalization_method = "each_time_bin"
fr_normalization_method = "across_time_bins"
plot_savedir = "/tmp"
dfallpa_preprocess_fr_normalization(DFallpa, fr_normalization_method, plot_savedir)

### (1b) FR normalization, apply across events


In [ ]:

# Prune to chans that are common across pa for each bregion (intersection of chans)|
dfpa_match_chans_across_pa_each_bregion(DFallpa)


##### Method 2 - Concat events (for each bregion) and normalize all same way


In [ ]:
from neuralmonkey.classes.population_mult import dfpa_concat_normalize_fr_split_multbregion
# fr_normalization_method = "each_time_bin"
fr_normalization_method = "across_time_bins"
dfpa_concat_normalize_fr_split_multbregion(DFallpa)


##### Method 3 - concat events (flexible version, only constrianed to have same n chans across PA)

In [ ]:
from neuralmonkey.classes.population_mult import dfpa_concat_normalize_fr_split_multbregion_flex
fr_mean_subtract_method = "across_time_bins"
# fr_mean_subtract_method = "each_time_bin"
PLOT=False

pa = DFallpa["pa"].values[0]
pa.plotNeurHeat(0)

dfpa_concat_normalize_fr_split_multbregion_flex(DFallpa, fr_mean_subtract_method, PLOT)

pa = DFallpa["pa"].values[0]
pa.plotNeurHeat(0)

### PCA

### Run for a single event

##### (1) PCA using trial data

In [ ]:
bregion = "PMv"
event="06_on_strokeidx_0"
PA = extract_single_pa(DFallpa, bregion, None, event=event)

In [ ]:
tbin_dur = 0.1
tbin_slide = 0.01
npcs_keep_force = 15
if True:
    # To plot things for each pc run
    plot_pca_explained_var_path = f"/tmp/plot_pca_explained_var_path-{bregion}.pdf"
    plot_loadings_path = f"/tmp/plot_loadings_path-{bregion}.pdf"
else:
    plot_pca_explained_var_path = None
    plot_loadings_path = None
norm_subtract_single_mean_each_chan = True
_, PAfinal1, _, _, _ = PA.dataextract_state_space_decode_flex(None, tbin_dur, tbin_slide, 
                                                        "chans_x_trials_x_times",
                                                        pca_reduce=True,
                                                        plot_pca_explained_var_path=plot_pca_explained_var_path, 
                                                        plot_loadings_path=plot_loadings_path,
                                                        norm_subtract_single_mean_each_chan=norm_subtract_single_mean_each_chan,
                                                        npcs_keep_force=npcs_keep_force)
plt.close("all")


##### (2) PCA on trial-averaged data

In [ ]:
import os

In [ ]:
savedir_plots = f"/tmp/pca_plots"
os.makedirs(savedir_plots, exist_ok=True)

var_pca = "seqc_0_shape"
vars_grouping = None
n_pcs_subspace_max = 15

# Fixed params
raw_subtract_mean_each_timepoint = False
pca_subtract_mean_each_level_grouping = True
n_min_per_lev_lev_others = 5
prune_min_n_levs = 2

# PCA time window -- use specific smaller window for fitting PC
pca_twind = (0.1, 1.0)
pca_tbindur = 0.1
pca_tbin_slice = 0.01

# Data projection time window -- keep larger window for data
proj_twind = (-0.5, 1.0)
proj_tbindur = 0.1
proj_tbin_slice = 0.01

Xredu, PAfinal2, stats_redu, Xfinal_before_redu, pca, PAraw = PA.dataextract_pca_demixed_subspace(var_pca, vars_grouping,
                                                pca_twind, pca_tbindur,
                                                savedir_plots=savedir_plots,
                                                raw_subtract_mean_each_timepoint=raw_subtract_mean_each_timepoint,
                                                pca_subtract_mean_each_level_grouping=pca_subtract_mean_each_level_grouping,
                                                n_min_per_lev_lev_others=n_min_per_lev_lev_others, prune_min_n_levs = prune_min_n_levs,
                                                n_pcs_subspace_max = n_pcs_subspace_max,
                                                do_pca_after_project_on_subspace=False,
                                                PLOT_STEPS=False, SANITY=False,
                                                reshape_method="chans_x_trials_x_times",
                                                pca_tbin_slice=pca_tbin_slice, return_raw_data=True,
                                                proj_twind = proj_twind, proj_tbindur = proj_tbindur, 
                                                proj_tbin_slice = proj_tbin_slice)



In [ ]:
# Comapre the results
# (i) raw data, pick a random channel
chan = PA.Chans[0]
PA.plotwrapper_smoothed_fr_split_by_label_and_subplots(chan, var_pca, ["seqc_0_loc"])

for pc in range(8):
    # (ii) PCa on trials
    PAfinal1.plotwrapper_smoothed_fr_split_by_label_and_subplots(pc, var_pca, ["seqc_0_loc"])
    # (iii) PCa on mean data
    PAfinal2.plotwrapper_smoothed_fr_split_by_label_and_subplots(pc, var_pca, ["seqc_0_loc"])

##### Run a single PCA across all events

In [ ]:
from neuralmonkey.classes.population_mult import dfpa_concat_pca_split_multbregion


In [ ]:

# Method 1 -- do PCA on trial-level data
sm_dur=0.1
sm_slide=0.01
npcs_keep_force=15
pca_method = "trials"
dfpa_concat_pca_split_multbregion(DFallpa, sm_dur=sm_dur, sm_slide=sm_slide,
                                      npcs_keep_force=npcs_keep_force,
                                      pca_method = pca_method)

In [ ]:
# Method 2 -- take mean over trials (--> one datapt per level of pcamean_var) and then do pca
sm_dur=0.1
sm_slide=0.01
npcs_keep_force=15
pca_method = "trial_means"
pcamean_var = "seqc_0_shape"
pcamean_vars_grouping = None
pcamean_pca_twind = (0.1, 1.2) 
dfpa_concat_pca_split_multbregion(DFallpa, sm_dur=sm_dur, sm_slide=sm_slide,
                                      npcs_keep_force=npcs_keep_force,
                                      pca_method = pca_method, 
                                      pcamean_var = pcamean_var, 
                                      pcamean_vars_grouping = pcamean_vars_grouping,
                                      pcamean_pca_twind = pcamean_pca_twind)

In [ ]:
# Look at raw and PCA data for an example PA
ind =1
pa = DFallpa["pa"].values[ind]
pa_pca = DFallpa["pa_pca"].values[ind]

pa.plotNeurHeat(0)
pa_pca.plotNeurHeat(0)


In [ ]:
# Visaulize PC encoding of shape.
for pc in range(6):
    pa_pca.plotwrapper_smoothed_fr_split_by_label_and_subplots(pc, var_pca, ["seqc_0_loc"])


### [Optionally] get a single PA concatted across events (for each bregion)


In [ ]:
from neuralmonkey.classes.population_mult import dfpa_group_and_split
DFallpaConcat = dfpa_group_and_split(DFallpa, ["event"], concat_dim="times", pa_column="pa_pca")

### Plot strokes (behavior) on this trial, overlaid on the task image

In [ ]:
dflab = pa.Xlabels["trials"]

Tkbeh_stkbeh = dflab.iloc[trial]["Tkbeh_stkbeh"]
Tkbeh_stktask = dflab.iloc[trial]["Tkbeh_stktask"]

strokes_beh = [tk["Prim"].Stroke() for tk in Tkbeh_stkbeh.Tokens]
strokes_task = [tk["Prim"].Stroke() for tk in Tkbeh_stktask.Tokens]

fig, ax = plt.subplots()
from pythonlib.drawmodel.strokePlots import plotDatStrokesWrapper, plotDatStrokes

# # Plot the task strokes
# plotDatStrokes(strokes_task, ax, mark_stroke_onset=False, color="k")

# # Overlay each beh stroke
# plotDatStrokes(strokes_beh, ax, mark_stroke_onset=True, add_stroke_number=True)

# Plot the task strokes
plotDatStrokes(strokes_task, ax, clean_task=True)

# Overlay each beh stroke
plotDatStrokes(strokes_beh, ax, clean_ordered_ordinal=True)


##### PA has many helper functions to preprocess and plot the data. 

In [ ]:
# Pick a random channel
chan = pa.Chans[4]


In [ ]:
# Plot heatmap of data for a single trial
trial = pa.Trials[0]
pa.plotNeurHeat(trial)

In [ ]:
# Plot average acrtivit for each shape
fig, ax = plt.subplots()
pa.plotwrapper_smoothed_fr_split_by_label("trials", "seqc_0_shape", ax, chan=chan)


fig, ax = plt.subplots()
pa.plotwrapper_smoothed_fr_split_by_label("trials", "seqc_0_shape", ax, chan=chan, plot_indiv=True)


In [ ]:
# Example processing. Here this picks out just a smaller time winodw
print("Times, before slicing: ", pa.Times)

twind = [-0.35, -0.3] # window: to only keep times within this window
pa_slice = pa.slice_by_dim_values_wrapper("times", twind)

print("Times, after slicing:", pa_slice.Times)

In [ ]:
# 1. First slice to time winodw, then take mean over time, i.e. a single time average
twind = (0.1, 0.5)
pa_sliced = pa.slice_by_dim_values_wrapper("times", twind)
pa_mean = pa_sliced.agg_wrapper("times")

print("Compare and see that times are binned")
print("-- original:", pa.Times, "... data shape: ", pa.X.shape)
print("-- sliced:", pa_sliced.Times, "... data shape: ", pa_sliced.X.shape)
print("-- mean:", pa_mean.Times, "... data shape: ", pa_mean.X.shape)


In [ ]:
# 2. binning time windows
binsize = 0.1
binslide = 0.02
pa_binned = pa.agg_by_time_windows_binned(binsize, binslide)
    
print("Compare and see that times are binned")
print("-- original:", pa.Times, "... data shape: ", pa.X.shape)
print("-- sliced:", pa_binned.Times, "... data shape: ", pa_binned.X.shape)


# Decoding shapes

This step takes in a representation of neural data and outputs a scalar score for how well you can decode "shape" from that data

Here, this example is using the raw data (dimensionality = number of channels). Could also use methods to reduce the dimensionality of this data, each time running through this decoding benchmark, to compare the different methods

### Make sure to normalize PA before running any modeling on it:

In [ ]:
from neuralmonkey.classes.population_mult import dfallpa_preprocess_fr_normalization
# fr_normalization_method = "each_time_bin"
fr_normalization_method = "across_time_bins"
plot_savedir = "/tmp"
dfallpa_preprocess_fr_normalization(DFallpa, fr_normalization_method, plot_savedir)

In [ ]:
# list_panorm = []
# for pa in DFallpa["pa"].tolist():
#     from neuralmonkey.analyses.state_space_good import popanal_preprocess_scalar_normalization
#     PAnorm, PAscal, PAscalagg, fig, axes, groupdict = popanal_preprocess_scalar_normalization(pa, None, DO_AGG_TRIALS=False)
#     list_panorm.append(PAnorm)
# DFallpa["pa"] = list_panorm
# if "pa_norm" in DFallpa.columns:
#     del DFallpa["pa_norm"]

### PREPROCESS - factorize all relevant labels FIRST here.


In [ ]:
from neuralmonkey.analyses.decode_good import preprocess_factorize_class_labels_ints
MAP_LABELS_TO_INT = preprocess_factorize_class_labels_ints(DFallpa)

### 1) Default: Time-resolved decoding

In [ ]:
# Simplest model, simply try to decode a single variable at each time bin, without caring about other possibly correlated variables.

In [ ]:
SAVEDIR_ANALYSIS = "/tmp/decoding"
import os

In [ ]:
SAVEDIR = f"{SAVEDIR_ANALYSIS}/1_time_resolved"
os.makedirs(SAVEDIR, exist_ok=True)
print(SAVEDIR)

n_min_trials = 6

from neuralmonkey.analyses.decode_good import decodewrap_categorical_timeresolved_singlevar, decodewrapouterloop_categorical_timeresolved

# list_vars_decode = ["seqc_0_shape", "seqc_0_loc"]
# list_vars_decode = ["seqc_0_shape"]
list_vars_decode = ["seqc_0_shape"]
# list_vars_decode = ["seqc_2_shape"]
# list_vars_decode = ["shape_is_novel_all"]

# PARAMS
time_bin_size = 0.2 # binsize, to average over to get population vector of neural activity
slide = 0.2 # to slide the bin.
max_nsplits = 2 # n times to do cross-validation.

DFRES = decodewrapouterloop_categorical_timeresolved(DFallpa, list_vars_decode, SAVEDIR, time_bin_size, slide, n_min_trials,
                                                     max_nsplits=max_nsplits)


### 2) Cross-condition decoding

In [ ]:
# Ask whether a decoder for variable X, trained on each level of variable Y, can generalize to the other levels of variable Y.

In [ ]:
# For each time bin, decode shape genearlizing across location

SAVEDIR = f"{SAVEDIR_ANALYSIS}/2_cross_condition_decoding"
os.makedirs(SAVEDIR, exist_ok=True)
print(SAVEDIR)

from neuralmonkey.analyses.decode_good import decodewrap_categorical_timeresolved_cross_condition, decodewrapouterloop_categorical_timeresolved_cross_condition

filtdict = None
separate_by_task_kind = True

# PARAMS

# # Context
# list_var_decode = [
#     "seqc_0_shape",
#     "shape-fixation",
#     "is-fixated-on-seqc0shape",
#     "is-fixated-on-seqc0shape",
#     "is-fixated-on-seqc0shape",
#     "is-fixated-on-seqc0shape",
# ]
# list_vars_conj = [
#     ["shape-macrosaccade-index", "seqc_0_loc_on_clust", "shape-fixation", "loc-fixation", "task_kind"],
#     ["shape-macrosaccade-index", "loc-fixation", "seqc_0_shape", "seqc_0_loc_on_clust", "task_kind"],
#     ["shape-macrosaccade-index", "seqc_0_shape", "loc-fixation", "task_kind"],
#     ["shape-macrosaccade-index", "seqc_0_shape", "seqc_0_loc_on_clust", "task_kind"],
#     ["shape-macrosaccade-index", "shape-fixation", "loc-fixation", "task_kind"],
#     ["shape-macrosaccade-index", "shape-fixation", "seqc_0_loc_on_clust", "task_kind"],
# ]
# Context
list_var_decode = [
    "seqc_0_shape",
    "seqc_0_loc",
]
list_vars_conj = [
    ["task_kind", "seqc_0_loc"],
    ["task_kind", "seqc_0_shape"],
]

time_bin_size = 0.2
slide = 0.2
subtract_mean_vars_conj = False
DFRES = decodewrapouterloop_categorical_timeresolved_cross_condition(DFallpa, list_var_decode,
                                                     list_vars_conj,
                                                     SAVEDIR, time_bin_size=time_bin_size, slide=slide,
                                                     subtract_mean_vars_conj=subtract_mean_vars_conj,
                                                                     filtdict=filtdict,
                                                                     separate_by_task_kind=separate_by_task_kind)



### 2b) Within-condition decoding. Separate decoder for each level of other var (then take average over decoders). Useful to controlling for variables

In [ ]:
# Decode variable X by building and scoring separate decoders conditioned on each level of variable Y.

In [ ]:
from neuralmonkey.analyses.decode_good import decodewrapouterloop_categorical_timeresolved_within_condition

SAVEDIR = f"{SAVEDIR_ANALYSIS}/3_within_condition_decoding"
os.makedirs(SAVEDIR, exist_ok=True)
print(SAVEDIR)

# PARAMS
separate_by_task_kind = True

# RUns
max_nsplits = 2
time_bin_size = 0.2
slide = 0.2

# PARAMS
DFRES = decodewrapouterloop_categorical_timeresolved_within_condition(DFallpa, list_var_decode,
                                                     list_vars_conj,
                                                    SAVEDIR, time_bin_size=time_bin_size, slide=slide, filtdict=filtdict,
                                                                      separate_by_task_kind=separate_by_task_kind)


# Dim reduction and State space plots

In [ ]:
SAVEDIR_ANALYSIS = "/tmp/Diego_230630"

### Plot scalar values (time-mean within twindow)

In [ ]:
from neuralmonkey.analyses.state_space_good import trajgood_construct_df_from_raw, trajgood_plot_colorby_splotby, trajgood_plot_colorby_splotby_scalar
from pythonlib.tools.plottools import savefig
from pythonlib.globals import PATH_ANALYSIS_OUTCOMES
import matplotlib.pyplot as plt
import os
from neuralmonkey.analyses.state_space_good import dimredgood_nonlinear_embed_data

### USER PARAMS
tbin_dur = 0.1
tbin_slide = 0.1
umap_n_neighbors = 45
pca_frac_var_keep = 0.8
n_min_per_levo= 5
list_twind_overall = [
    [-0.3, 0.0],
    [0.0, 0.3]
]
# METHOD = "umap"
METHOD = "umap"
list_var_color_var_subplot = [
    ["seqc_0_shape", ("seqc_0_loc", "task_kind")],
]

LIST_DIMS = [(0,1)]

### HARD PARAMS
reshape_method = "trials_x_chanstimes"

for i, row in DFallpa.iterrows():
    pa = row["pa"]
    br = row["bregion"]
    wl = row["which_level"]
    ev = row["event"]
    tw = row["twind"]

    for twind_overall in list_twind_overall:
        
        ###################### SCALAR DATA
        # Extract data
        X, _, pathis, _, _  = pa.dataextract_state_space_decode_flex(twind_overall, tbin_dur, tbin_slide, reshape_method,
                                                           pca_reduce=True, pca_frac_var_keep=pca_frac_var_keep)
        dflab = pathis.Xlabels["trials"]
        
        if METHOD=="umap":
            # Embed data
            Xredu, _ = dimredgood_nonlinear_embed_data(X, METHOD=METHOD, n_components=2, umap_n_neighbors=umap_n_neighbors)
        elif METHOD=="pca":
            Xredu = X
        else:
            print(METHOD)
            assert False

        ##### Plot scalars
        savedir = f"{SAVEDIR_ANALYSIS}/scalar/{br}-{ev}-twind={'_'.join([str(t) for t in twind_overall])}-METHOD={METHOD}"
        print(savedir)
        os.makedirs(savedir, exist_ok=True)

        for var_color, var_subplot in list_var_color_var_subplot:
            from neuralmonkey.analyses.state_space_good import trajgood_plot_colorby_splotby_scalar_WRAPPER
            trajgood_plot_colorby_splotby_scalar_WRAPPER(Xredu, dflab, var_color, savedir,
                                     vars_subplot=var_subplot, list_dims=LIST_DIMS, n_min_per_levo=n_min_per_levo)
        plt.close("all")

### Plot trajectories (within time window)

In [ ]:
from neuralmonkey.analyses.state_space_good import trajgood_construct_df_from_raw, trajgood_plot_colorby_splotby, trajgood_plot_colorby_splotby_scalar
from pythonlib.tools.plottools import savefig
from pythonlib.globals import PATH_ANALYSIS_OUTCOMES
import os
from neuralmonkey.analyses.state_space_good import dimredgood_nonlinear_embed_data
import matplotlib.pyplot as plt

### USER PARAMS
tbin_dur = 0.1
tbin_slide = 0.1
umap_n_neighbors = 45
pca_frac_var_keep = 0.8
n_min_per_levo= 5
list_twind_overall = [
    [0.2, 0.6]
]
# METHOD = "umap"
METHOD = "pca"
list_var_color_var_subplot = [
    ["seqc_0_shape", ("seqc_0_loc", "task_kind")],
]
LIST_DIMS = [(0,1)]

### HARD PARAMS
reshape_method = "chans_x_trials_x_times"


for i, row in DFallpa.iterrows():
    pa = row["pa"]
    br = row["bregion"]
    wl = row["which_level"]
    ev = row["event"]
    tw = row["twind"]

    for twind_overall in list_twind_overall:
        
        ###################### TRAJECTORY DATA
        from neuralmonkey.analyses.state_space_good import trajgood_construct_df_from_raw, trajgood_plot_colorby_splotby
        from neuralmonkey.classes.population_mult import extract_single_pa
        twind_overall_traj = [-0.2, 0.6]
        tbin_dur = 0.05
        X, PApca, PAslice, pca, _ = pa.dataextract_state_space_decode_flex(twind_overall_traj, tbin_dur=tbin_dur, pca_reduce=True)
        
        if METHOD=="pca":
            Xredu = X
        else:
            print(METHOD)
            assert False, "code it"
            
        savedir = f"{SAVEDIR_ANALYSIS}/trajectories/{br}-{ev}-twind={'_'.join([str(t) for t in twind_overall_traj])}-METHOD=pca"
        print(savedir)
        os.makedirs(savedir, exist_ok=True)
        
        for var_color, var_subplot in list_var_color_var_subplot:
            for dims in LIST_DIMS:
                # 1) Construct dataframe
                if isinstance(var_color, (list, tuple)):
                    grpvars = list(var_color) + list(var_subplot)
                else:
                    grpvars = [var_color] + list(var_subplot)
                    
                labels = PApca.Xlabels["trials"].loc[:, grpvars] 
                labelvars = grpvars
                df = trajgood_construct_df_from_raw(X, PApca.Times, labels, labelvars)
                
                # 2) Plot
                # dims = (0,1) # pairs of dimensions to plot
                times_to_mark = [0.] # you can mark specific times on the plot. here marks the 0. sec mark.
                times_to_mark_markers = ["d"] # mark with a diamond ("d")
                time_bin_size = 0.05 # to make plot easier to visaulize, you can bin in time.
                
                fig, axes = trajgood_plot_colorby_splotby(df, var_color, var_subplot, dims, "traj", mean_over_trials=True,
                                              times_to_mark = times_to_mark, times_to_mark_markers = times_to_mark_markers,
                                              time_bin_size=time_bin_size)        
                
                path = f"{savedir}/color={var_color}-sub={var_subplot}-dims={dims[0], dims[1]}.pdf"
                print("Saving ... ", path)
                savefig(fig, path)
                
                plt.close("all")
            

In [ ]:
sorted(dflab.columns.tolist())

# Wrapper to make all state space plots

In [ ]:
pa = DFallpa["pa"].values[0]
pa.Xlabels["trials"]

In [ ]:
# (1) Dim reduction
# (2) Plots.


In [ ]:
# Sources:
# - analy_state_space_... (script)
# - euclidian_plot (scalar)
# - euclidna (traj)
# - eye fixation state space.
# - above, tutorial.



### Dim reduction

##### Iterate (unit test)

In [ ]:
dpca_var = "seqc_0_shape"
dpca_vars_group = ["seqc_0_loc"]
dpca_filtdict={"epoch":["SSD3"]}
dpca_proj_twind = None
umap_n_components = 3

for dim_red_method in [None, "pca", "pca_umap", "umap", "mds", "dpca"]:
    for scalar_or_traj in ["scal", "traj"]:
        # dim_red_method = None
        # scalar_or_traj = "scalar"

        savedir = f"/tmp/{dim_red_method}-{scalar_or_traj}"
        os.makedirs(savedir, exist_ok=True)

        twind_pca = (0.1, 0.5)
        tbin_dur = 0.2
        tbin_slide = 0.2

        Xredu, PAredu = PA.dataextract_dimred_wrapper(scalar_or_traj, dim_red_method, savedir, 
                                        twind_pca, tbin_dur, tbin_slide, 
                                        NPCS_KEEP = 10,
                                        dpca_var = dpca_var, dpca_vars_group = dpca_vars_group, dpca_filtdict=dpca_filtdict, dpca_proj_twind = dpca_proj_twind,
                                        umap_n_components =umap_n_components,
                                        raw_subtract_mean_each_timepoint=False)
        plt.close("all")
        print(" ====> ", savedir, Xredu.shape, PAredu.X.shape)

##### Get a single case

In [7]:
from neuralmonkey.scripts.analy_pig_decode_moment_syntaxTI import dfallpa_preprocess_condition
dfallpa_preprocess_condition(DFallpa)

In [8]:
pa = DFallpa["pa"].values[0]

In [9]:
# only successful trials
pa = DFallpa["pa"].values[0]
dflab = pa.Xlabels["trials"]
shapes_good = dflab["shape"].value_counts()[dflab["shape"].value_counts()>10].index.tolist()
trialcode_good = dflab[dflab["FEAT_num_strokes_task"] == dflab["FEAT_num_strokes_beh"]]["trialcode"].tolist()
trialcode_fail = dflab[dflab["FEAT_num_strokes_task"] > dflab["FEAT_num_strokes_beh"]]["trialcode"].tolist()


In [ ]:
import os

bregion = "M1_m"
raw_subtract_mean_each_timepoint = False
scalar_or_traj = "scal"

which_level = "stroke"
event = "00_stroke"
twind_pca = (0, 0.3)

# event = "03_samp"
# twind_pca = (0, 1.6)

# event = "06_on_strokeidx_0"
# twind_pca = (-0.15, 0.35)

dpca_filtdict={
    # "seqc_0_shape":shapes_good,
    "shape":shapes_good,
    "trialcode":trialcode_good,
    # "task_kind":["prims_on_grid"],
    "success_binary_quick":[True],
    # "trialcode":trialcode_fail,trialcode_fail
    }

dpca_var = "shape"
dpca_vars_group = ["stroke_index"]

# dpca_var = "seqc_0_shape"
# dpca_vars_group = None

# dpca_var = "seqc_1_shape"
# dpca_vars_group = ["seqc_0_shape"]

# dpca_filtdict={"epoch":["SSD3"]}
# dpca_filtdict=None

dpca_proj_twind = None
umap_n_components = 3

tbin_dur = 0.1
tbin_slide = 0.05

dim_red_method = "dpca"

savedir = f"/tmp/TESTTHIS"
os.makedirs(savedir, exist_ok=True)

PA = extract_single_pa(DFallpa, bregion, None, which_level, event)


Xredu, PAredu = PA.dataextract_dimred_wrapper(scalar_or_traj, dim_red_method, savedir, 
                                twind_pca, tbin_dur, tbin_slide, 
                                NPCS_KEEP = 10,
                                dpca_var = dpca_var, dpca_vars_group = dpca_vars_group, dpca_filtdict=dpca_filtdict, dpca_proj_twind = dpca_proj_twind,
                                umap_n_components =umap_n_components,
                                raw_subtract_mean_each_timepoint=raw_subtract_mean_each_timepoint)


### Plots

In [ ]:
savedir = f"/tmp/{animal}-{date}-PLOTS6-{bregion}-{event}-{dim_red_method}-{dpca_var}-{raw_subtract_mean_each_timepoint}-2"
print(savedir)
os.makedirs(savedir, exist_ok=True)


In [ ]:
# LIST_VAR = ["seqc_0_shape", "seqc_0_shape", "seqc_0_shape", "seqc_1_shape", "seqc_1_shape"]
# LIST_VARS_OTHERS = [
#     ("task_kind", "FEAT_num_strokes_task", "success_binary_quick",), 
#     ("task_kind", "success_binary_quick",), 
#     ("task_kind",),
#     ("task_kind", "FEAT_num_strokes_task", "success_binary_quick",), 
#     ("task_kind", "FEAT_num_strokes_task", "seqc_0_shape", "success_binary_quick",), 
#     ]
# LIST_FILTDICT = [
#     {"seqc_0_shape":shapes_good} for _ in range(len(LIST_VAR))
# ]

LIST_VAR = ["shape", "shape",  "shape", "shape", "shape", "stroke_index", "stroke_index"]
LIST_VARS_OTHERS = [
    ("task_kind", "FEAT_num_strokes_task", "success_binary_quick", "stroke_index"), 
    ("task_kind", "FEAT_num_strokes_task", "success_binary_quick"), 
    ("task_kind", "FEAT_num_strokes_task"), 
    ("task_kind", "stroke_index", "success_binary_quick"), 
    ("task_kind", "stroke_index"), 
    ("task_kind", "FEAT_num_strokes_task", "shape"), 
    ("task_kind", "FEAT_num_strokes_task"), 
    ]
LIST_FILTDICT = [
    {"shape":shapes_good} for _ in range(len(LIST_VAR))
]

PLOT_CLEAN_VERSION = False
time_bin_size = 0.05
PAredu.plot_state_space_good_wrapper(savedir, LIST_VAR, LIST_VARS_OTHERS, LIST_FILTDICT, 
                                     PLOT_CLEAN_VERSION=PLOT_CLEAN_VERSION,
                                     time_bin_size=time_ n h   

In [ ]:
savedir = "/tmp/PLOTS_SCAL_2"
os.makedirs(savedir, exist_ok=True)

PAredu_scal = PAredu.agg_wrapper("times")
PAredu_scal.plot_state_space_good_wrapper(savedir, LIST_VAR, LIST_VARS_OTHERS, PLOT_CLEAN_VERSION=PLOT_CLEAN_VERSION)